In [ ]:
!pip install influxdb-client

In [ ]:
from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

def query_coffee_history():
    # You can generate an API token from the "API Tokens Tab" in the UI
    token = "Y2nr_V73MfQqmotuaeReBnRnuPDihdKm-RCSFRwfyFhlxx4b7YjUkzBokDrhV3q6G2ib-Arknf6e9itL9ROJDg=="
    org = "uiam"
    bucket = "mqtt"

    with InfluxDBClient(url="https://influxdb.cloud.uiam.sk", token=token, org=org) as client:
        query = '''
        from(bucket: "mqtt") 
            |> range(start: 2021-05-22T23:30:00Z) 
            |> filter(fn: (r) => r._measurement == "shellies" and r._field == "power" and r.device == "Shelly_Kitchen-C_CoffeMachine/relay/0")'''
        tables = client.query_api().query(query, org=org)
        client.close()
    
    l = []
    for table in tables:
        for record in table.records:
            l.append({'time': str(record.get_time()),
                      'power': record.get_value()})
        
    return l

In [ ]:
l = query_coffee_history()

In [ ]:
import json
with open("coffee_history.json", "w") as final:
   json.dump(l, final)

# MQTT

In [ ]:
!pip install paho-mqtt

In [ ]:
#!git clone https://github.com/eclipse/paho.mqtt.python

In [ ]:
broker = "mqtt.cloud.uiam.sk"
port = 1883
topics = ["shellies/Shelly_Kitchen-C_CoffeMachine/relay/0/power"]
client_id = "Smartphone"
username = 'admin'
password = 'kefxo5-kugjEj-qodboc'
auth = {'username': username, 'password': password}

In [ ]:
import paho.mqtt.subscribe as subscribe

topics = ['shellies/#']
topics = ["shellies/Shelly_Kitchen-C_CoffeMachine/relay/0/power"]

m = subscribe.simple(topics, hostname=broker, retained=False, msg_count=2, client_id=client_id, clean_session=True)
for a in m:
    print(a.topic)
    print(a.timestamp)
    print(a.payload.decode())

In [ ]:
float(a.payload.decode())

In [ ]:
import paho.mqtt.subscribe as subscribe

def print_msg(client, userdata, message):
    print("%s : %s" % (message.topic, message.payload))

subscribe.callback(print_msg, topics, hostname=broker, client_id=client_id, clean_session=True)

In [ ]:
import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(topics)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload.decode()))

client = mqtt.Client(client_id, clean_session=True)
client.on_connect = on_connect
client.on_message = on_message

client.connect(broker, port, keepalive=6)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

# Sandbox

In [ ]:
import json
file = open('coffee_history.json')
data_cof = json.load(file)

In [ ]:
import pandas as pd
from river import linear_model, metrics, evaluate, preprocessing, datasets, optim, imblearn, anomaly, compose
from streamz import Stream
#####################################################
model_batch_LR = (
    preprocessing.StandardScaler() |
    linear_model.LogisticRegression()
)

metric_batch_LR = metrics.F1()


#####################################################
model_strm_LR = (
    preprocessing.StandardScaler() |
    linear_model.LogisticRegression()
)

metric_strm_LR = metrics.F1()



##### teach batch models
batch_size = 12500
#batch_size = 20000

x = pd.DataFrame(data_cof[0:batch_size])
model_batch_LR.learn_many(x, x['power'] > 200)

for row in data_cof[0:batch_size]:
  y = 1 if row['power'] > 200 else 0
  score = model_strm_LR.predict_one(row)
  #model_strm_LR.quantile.update(score)
  #print(y)
  #pred = model_strm_LR.predict_one(row)
  #print(pred)
  model_strm_LR.learn_one(row, y)
  #model_batch_SVM.learn_one(row)
  ### tu sa z nejakeho bud uci zle, alebo mam zle implementovanu streamz pipelinu

In [ ]:
for row in data_cof[batch_size:-1]:
    y = 1 if row['power'] > 200 else 0
    y_pred = model_batch_LR.predict_one(row)
    metric_batch_LR.update(y, y_pred)
    y_pred = model_strm_LR.predict_one(row)
    metric_strm_LR.update(y, y_pred)

In [ ]:
metric_batch_LR

In [ ]:
metric_strm_LR

In [ ]:
model_batch_LR.predict_one(data_cof[batch_size+1])

In [ ]:
model_strm_LR.predict_one(data_cof[batch_size+1])

# MQTT with Streamz

In [ ]:
from streamz import Stream

In [ ]:
def process_coffee_present(x):
    return {'power': x.payload.decode()}

In [ ]:
topic = "shellies/Shelly3EM-Main-Switchboard-C/emeter/0/energy"

In [ ]:
s = Stream.from_mqtt(broker, port, topics[0])

s.map(process_coffee_present).sink(print)

In [ ]:
s.start()

In [ ]:
s.stop()